# Homework - Grover MaxCut

The places where you have enter code are marked with `# YOUR CODE HERE`.

In [3]:
import cirq
from cirq import H, X, Y, Z, CX, inverse

## Question 1 (4 points)

Write a function, `oracle010`, that implements an oracle that marks the state $|010 \rangle$. The function `oracle010` has

* input: `qq`, a 3-qubit register 
* returns: None

The function should append a sequence of gates to `qq` to mark the state $|010\rangle$ only. Don't append any measurements to `qq`.

To help you test the function, we have provided the `grover_diffusion` and `grover` functions.

In [8]:
def oracle010(qq):
    # Define the target state to be marked (|010⟩)
    target_state = [0, 1, 0]

    # Iterate through the qubits and apply X gates to flip them when necessary
    for i, bit in enumerate(target_state):
        if bit == 1:
            # Apply an X gate to flip the qubit to |1⟩
            qq.append(cirq.X(qq[i]))

In [9]:
# visualize your implemented gates
qqTest = cirq.LineQubit.range(3)
circuit = cirq.Circuit()
circuit.append(oracle010(qqTest))
circuit

TypeError: Not an Operation or Iterable: <class 'NoneType'> None

In [17]:
# To check your solution, we need some to implement grover
def grover_diffusion(qq,n):
    yield H.on_each(*qq)
    yield X.on_each(*qq)
    yield Z(qq[n-1]).controlled_by(*(qq[0:n-1]))
    yield X.on_each(*qq)
    yield H.on_each(*qq)

In [18]:
def grover(trials_number):
    n=3
    qq = cirq.LineQubit.range(n)
    circuit = cirq.Circuit()
    circuit.append(H.on_each(*qq))  

    for i in range(2):
        circuit.append(oracle010(qq))
        circuit.append(grover_diffusion(qq,n))
    circuit.append(cirq.measure(*qq, key='result'))

    # determine the statistics of the measurements
    s = cirq.Simulator() 
    
    samples = s.run(circuit, repetitions=trials_number)

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    print(counts)
    return counts.get('010')

In [19]:
# run grover to test if your function gives the right answer
grover(100)

Counter({'011': 21, '111': 14, '101': 13, '000': 13, '001': 11, '100': 10, '110': 10, '010': 8})


8

In [ ]:
# hidden tests in this cell will be used for grading.

## Question 2 (6 points)

Write an oracle for the graph described below to check whether it admits a valid 2-coloring. 

The function `oracle2` has

* input: `qq`, a 12-qubit register 
* returns: None

The function should append a sequence of gates to `qq` to check for a valid coloring. Don't append any measurements to `qq`.

You can test the oracle with the provided `grover_diffusion`, `grover` and `oracle_computation2` functions.



$G$ has 5 vertices. Use qubits 0-4 for the vertices, 5-10 for the edges and 11 as the ancilla.

The list of edges:
(0,3)
(0,4)
(1,3)
(1,4)
(2,3)
(2,4)


In [20]:
def oracle2(qq):
    # Define the edges of the graph
    edges = [(0, 3), (0, 4), (1, 3), (1, 4), (2, 3), (2, 4)]
    
    # Apply gates to enforce the coloring constraint
    for edge in edges:
        vertex1, vertex2 = edge
        # Use the XOR operation to ensure that vertices connected by an edge have different colors
        yield cirq.CNOT(qq[vertex1], qq[vertex2])
    
    # Check if all edges satisfy the coloring constraint using an ancilla qubit
    yield cirq.X(qq[11]).controlled_by(*qq[5:11])

In [21]:
# We need some code so you can check your solution
def oracle_computation2(qq):
    yield oracle2(qq)
    yield Z(qq[11])
    yield inverse(oracle2(qq))  

In [22]:
def grover2(trials_number):    
    import cirq
    from cirq import X, H, Z, inverse, CX
    s = cirq.Simulator()

    qq = cirq.LineQubit.range(12)
    n=5
    
    circuit = cirq.Circuit()
    circuit.append(H.on_each(*(qq[0:n])))
    for i in range(2):
        circuit.append(oracle_computation2(qq))
        circuit.append(grover_diffusion(qq,n))

    circuit.append(cirq.measure(*(qq[0:n]), key='result'))

    # determine the statistics of the measurements
    samples = s.run(circuit, repetitions=trials_number)
    result = samples.measurements["result"]

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    return counts

In [23]:
#You can use this cell to test your solution
shots=1000
grover2(shots)

Counter({'00101': 35,
         '01011': 34,
         '00110': 31,
         '10100': 32,
         '00000': 37,
         '10101': 46,
         '01110': 38,
         '00011': 30,
         '00001': 25,
         '00100': 38,
         '11000': 24,
         '01010': 27,
         '10110': 33,
         '11101': 38,
         '00010': 33,
         '01100': 34,
         '11110': 33,
         '01001': 31,
         '01000': 33,
         '11001': 33,
         '01101': 29,
         '10010': 27,
         '11100': 33,
         '01111': 21,
         '10000': 19,
         '11010': 30,
         '00111': 31,
         '10011': 34,
         '10111': 26,
         '10001': 32,
         '11111': 25,
         '11011': 28})

In [ ]:
# hidden tests in this cell will be used for grading.

## Question 3 (10 points)

Write an oracle for the graph described below to check whether there exists a coloring with 4 edges connecting vertices with different colors.

The function `oracle3` has

* input: `qq`, a 14-qubit register 
* returns: None

The function should append a sequence of gates to `qq` to check whether there exists a coloring with 4 edges connecting vertices with different colors. Don't append any measurements to `qq`.

You can test the oracle with the provided `grover_diffusion`, `grover` and `oracle_computation3` functions.


$G$ has 4 vertices. Use qubits 0-4 for the vertices, 5-9 for the edges, 10-12 for the addition and 13 as the ancilla.

The list of edges: 
(0,1)
(0,2)
(0,3)
(1,2)
(1,3)

In [40]:
import cirq

def oracle3(qq):
    # Define the graph
    G = [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3)]
    vertices = [cirq.GridQubit(i, 0) for i in range(5)]
    edges = [cirq.GridQubit(i, j) for i in range(5) for j in range(1, 2)]
    ancilla = cirq.GridQubit(13, 0)

    # Append the sequence of gates to qq to check whether there exists a coloring with 4 edges connecting vertices with different colors
    yield cirq.X.on_each(vertices)
    yield cirq.H.on_each(edges)
    yield cirq.H.on_each(vertices)
    yield cirq.CNOT(vertices[0], edges[0])
    yield cirq.CNOT(vertices[0], edges[1])
    yield cirq.CNOT(vertices[0], edges[2])
    yield cirq.CNOT(vertices[1], edges[0])
    yield cirq.CNOT(vertices[1], edges[1])
    yield cirq.CNOT(vertices[1], edges[3])
    yield cirq.CNOT(vertices[2], edges[2])
    yield cirq.CNOT(vertices[2], edges[3])
    yield cirq.CNOT(vertices[2], edges[4])
    yield cirq.CNOT(vertices[3], edges[4])
    yield cirq.H.on_each(edges)
    yield cirq.X.on_each(vertices)



In [41]:
# We need some code so you can check your solution
def oracle_computation3(qq):
    yield oracle3(qq)
    yield Z(qq[13])
    yield inverse(oracle3(qq))  

In [42]:
import cirq
from cirq import X, H, Z, inverse, CX, CCX
    
def grover3(trials_number):    
    s = cirq.Simulator()

    qq = cirq.LineQubit.range(14)
    n=4

    circuit = cirq.Circuit()
    circuit.append(H.on_each(*(qq[0:n])))
    for i in range(2):
        circuit.append(oracle_computation3(qq))
        circuit.append(grover_diffusion(qq,n))

    circuit.append(cirq.measure(*(qq[0:n]), key='result'))

    # determine the statistics of the measurements
    samples = s.run(circuit, repetitions=trials_number)
    result = samples.measurements["result"]

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    return counts

In [27]:
#You can use this cell to test your solution
shots=1000
grover3(shots)

ValueError: Duplicate qids for <cirq.CNOT>. Expected unique qids but got <[cirq.LineQubit(13), cirq.LineQubit(13)]>.

In [ ]:
# hidden tests in this cell will be used for grading.